# X

---

### 1. What is X?

---

### 2. Why is X important?

---

### 3. What are some applications of X? What other concepts can I connect to X? Use FE when...

---

### 4. What is the evolution/history of X?

---

### 5. Can I predict the future use of X? How can this current usage of X improve?

---

### 6. What don't I understand about X? Why is this? What's the root of this misunderstanding?

---


## Imports and Load Data

In [1]:
import numpy as np

## Transform Time Series Problem to Supervised Learning Problem (My Solution)
- Function: `split_sequence()`
- Parameters:
    - `ts_data`: time series data to transform to supervised learning data
    - `input_X_size`: size of each `X` for supervised learning
- Input: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
- Output: X_1 = [1, 2, 3] Y_1 = [4]
X_2 = [2, 3, 4] Y_2 = [5]
X_3 = [3, 4, 5] Y_2 = [6]
...

In [28]:
def split_sequence(ts_data, input_X_size):
    """
    ts_data -- 1D np array
    input_X_size -- int
    
    return -- 2D np array, 1D np array
    """
    X = []
    y = []
    
    for i in range(len(ts_data)):
        
        last_idx = i + input_X_size
        print(ts_data[i], last_idx)
        
        if last_idx == len(ts_data):
            break
        
        # set the size of all X inputs
        get_X = ts_data[i : last_idx]
        print(get_X)
        
        X.append(get_X)
        print(X)
        
        get_y = ts_data[last_idx]
        print(get_y)
        
        y.append(get_y)
        print(y)
        
        
        print()
    return X, y

In [29]:
time_series_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
split_sequence(time_series_data, 3)

1 3
[1 2 3]
[array([1, 2, 3])]
4
[4]

2 4
[2 3 4]
[array([1, 2, 3]), array([2, 3, 4])]
5
[4, 5]

3 5
[3 4 5]
[array([1, 2, 3]), array([2, 3, 4]), array([3, 4, 5])]
6
[4, 5, 6]

4 6
[4 5 6]
[array([1, 2, 3]), array([2, 3, 4]), array([3, 4, 5]), array([4, 5, 6])]
7
[4, 5, 6, 7]

5 7
[5 6 7]
[array([1, 2, 3]), array([2, 3, 4]), array([3, 4, 5]), array([4, 5, 6]), array([5, 6, 7])]
8
[4, 5, 6, 7, 8]

6 8
[6 7 8]
[array([1, 2, 3]), array([2, 3, 4]), array([3, 4, 5]), array([4, 5, 6]), array([5, 6, 7]), array([6, 7, 8])]
9
[4, 5, 6, 7, 8, 9]

7 9
[7 8 9]
[array([1, 2, 3]), array([2, 3, 4]), array([3, 4, 5]), array([4, 5, 6]), array([5, 6, 7]), array([6, 7, 8]), array([7, 8, 9])]
10
[4, 5, 6, 7, 8, 9, 10]

8 10


([array([1, 2, 3]),
  array([2, 3, 4]),
  array([3, 4, 5]),
  array([4, 5, 6]),
  array([5, 6, 7]),
  array([6, 7, 8]),
  array([7, 8, 9])],
 [4, 5, 6, 7, 8, 9, 10])

## Transform Time Series Problem to Supervised Learning Problem (Correct Solution)